In [ ]:
#@title Imports & Utils

# Imports

!pip install -q git+https://www.github.com/google/jax-md

import numpy as onp

from jax.config import config ; config.update('jax_enable_x64', True)
import jax.numpy as np
from jax import random
from jax import jit
from jax import vmap
from jax import lax
vectorize = np.vectorize

from functools import partial

from collections import namedtuple
import base64

import IPython
from google.colab import output

import os

from jax_md import space, smap, energy, minimize, quantity, simulate, partition, util
from jax_md.util import f32

# Plotting

import matplotlib.pyplot as plt
import seaborn as sns
  
sns.set_style(style='white')

dark_color = [56 / 256] * 3
light_color = [213 / 256] * 3
axis_color = 'white'

def format_plot(x='', y='', grid=True):  
  ax = plt.gca()
  
  ax.spines['bottom'].set_color(axis_color)
  ax.spines['top'].set_color(axis_color) 
  ax.spines['right'].set_color(axis_color)
  ax.spines['left'].set_color(axis_color)
  
  ax.tick_params(axis='x', colors=axis_color)
  ax.tick_params(axis='y', colors=axis_color)
  ax.yaxis.label.set_color(axis_color)
  ax.xaxis.label.set_color(axis_color)
  ax.set_facecolor(dark_color)
  
  plt.grid(grid)
  plt.xlabel(x, fontsize=20)
  plt.ylabel(y, fontsize=20)
  
def finalize_plot(shape=(1, 1)):
  plt.gcf().patch.set_facecolor(dark_color)
  plt.gcf().set_size_inches(
    shape[0] * 1.5 * plt.gcf().get_size_inches()[1], 
    shape[1] * 1.5 * plt.gcf().get_size_inches()[1])
  plt.tight_layout()

# Progress Bars

from IPython.display import HTML, display
import time


def ProgressIter(iter_fun, iter_len=0):
  if not iter_len:
    iter_len = len(iter_fun)
  out = display(progress(0, iter_len), display_id=True)
  for i, it in enumerate(iter_fun):
    yield it
    out.update(progress(i + 1, iter_len))

def progress(value, max):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 45%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

normalize = lambda v: v / np.linalg.norm(v, axis=1, keepdims=True)

# Rendering

renderer_code = IPython.display.HTML('''
<canvas id="canvas"></canvas>
<script>
  Rg = null;
  Ng = null;

  var current_scene = {
      R: null,
      N: null,
      is_loaded: false,
      frame: 0,
      frame_count: 0,
      boid_vertex_count: 0,
      boid_buffer: [],
      predator_vertex_count: 0,
      predator_buffer: [],
      disk_vertex_count: 0,
      disk_buffer: null,
      box_size: 0
  };

  google.colab.output.setIframeHeight(0, true, {maxHeight: 5000});

  async function load_simulation() {
    buffer_size = 400;
    max_frame = 800;

    result = await google.colab.kernel.invokeFunction(
        'notebook.GetObstacles', [], {});
    data = result.data['application/json'];

    if(data.hasOwnProperty('Disk')) {
      current_scene = put_obstacle_disk(current_scene, data.Disk);
    }

    for (var i = 0 ; i < max_frame ; i += buffer_size) {
      console.log(i);
      result = await google.colab.kernel.invokeFunction(
          'notebook.GetBoidStates', [i, i + buffer_size], {}); 
      
      data = result.data['application/json'];
      current_scene = put_boids(current_scene, data);
    }
    current_scene.is_loaded = true;

    result = await google.colab.kernel.invokeFunction(
        'notebook.GetPredators', [], {}); 
    data = result.data['application/json'];
    if (data.hasOwnProperty('R'))
      current_scene = put_predators(current_scene, data);

    result = await google.colab.kernel.invokeFunction(
          'notebook.GetSimulationInfo', [], {});
    current_scene.box_size = result.data['application/json'].box_size;
  }

  function initialize_gl() {
    const canvas = document.getElementById("canvas");
    canvas.width = 640;
    canvas.height = 640;

    const gl = canvas.getContext("webgl2");

    if (!gl) {
        alert('Unable to initialize WebGL.');
        return;
    }

    gl.viewport(0, 0, gl.drawingBufferWidth, gl.drawingBufferHeight);
    gl.clearColor(0.2, 0.2, 0.2, 1.0);
    gl.enable(gl.DEPTH_TEST);

    const shader_program = initialize_shader(
        gl, VERTEX_SHADER_SOURCE_2D, FRAGMENT_SHADER_SOURCE_2D);
    const shader = {
      program: shader_program,
      attribute: {
          vertex_position: gl.getAttribLocation(shader_program, 'vertex_position'),
      },
      uniform: {
          screen_position: gl.getUniformLocation(shader_program, 'screen_position'),
          screen_size: gl.getUniformLocation(shader_program, 'screen_size'),
          color: gl.getUniformLocation(shader_program, 'color'),
      },
    };
    gl.useProgram(shader_program);

    const half_width = 200.0;

    gl.uniform2f(shader.uniform.screen_position, half_width, half_width);
    gl.uniform2f(shader.uniform.screen_size, half_width, half_width);
    gl.uniform4f(shader.uniform.color, 0.9, 0.9, 1.0, 1.0);

    return {gl: gl, shader: shader};
  }

  var loops = 0;

  function update_frame() {
    gl.clear(gl.COLOR_BUFFER_BIT | gl.DEPTH_BUFFER_BIT);

    if (!current_scene.is_loaded) {
      window.requestAnimationFrame(update_frame);
      return;
    }

    var half_width = current_scene.box_size / 2.;
    gl.uniform2f(shader.uniform.screen_position, half_width, half_width);
    gl.uniform2f(shader.uniform.screen_size, half_width, half_width);

    if (current_scene.frame >= current_scene.frame_count) {
      if (!current_scene.is_loaded) {
        window.requestAnimationFrame(update_frame);
        return;
      }
      loops++;
      current_scene.frame = 0;
    }

    gl.enableVertexAttribArray(shader.attribute.vertex_position);

    gl.bindBuffer(gl.ARRAY_BUFFER, current_scene.boid_buffer[current_scene.frame]);
    gl.uniform4f(shader.uniform.color, 0.0, 0.35, 1.0, 1.0);
    gl.vertexAttribPointer(
      shader.attribute.vertex_position,
      2,
      gl.FLOAT,
      false,
      0,
      0
    );
    gl.drawArrays(gl.TRIANGLES, 0, current_scene.boid_vertex_count);

    if(current_scene.predator_buffer.length > 0)  {
      gl.bindBuffer(gl.ARRAY_BUFFER, current_scene.predator_buffer[current_scene.frame]);
      gl.uniform4f(shader.uniform.color, 1.0, 0.35, 0.35, 1.0);
      gl.vertexAttribPointer(
        shader.attribute.vertex_position,
        2,
        gl.FLOAT,
        false,
        0,
        0
      );
      gl.drawArrays(gl.TRIANGLES, 0, current_scene.predator_vertex_count);
    }
    
    if(current_scene.disk_buffer) {
      gl.bindBuffer(gl.ARRAY_BUFFER, current_scene.disk_buffer);
      gl.uniform4f(shader.uniform.color, 0.9, 0.9, 1.0, 1.0);
      gl.vertexAttribPointer(
        shader.attribute.vertex_position,
        2,
        gl.FLOAT,
        false,
        0,
        0
      );
      gl.drawArrays(gl.TRIANGLES, 0, current_scene.disk_vertex_count);
    }

    current_scene.frame++;
    if ((current_scene.frame_count > 1 && loops < 5) || 
        (current_scene.frame_count == 1 && loops < 240))
      window.requestAnimationFrame(update_frame);
    
    if (current_scene.frame_count > 1 && loops == 5 && current_scene.frame < current_scene.frame_count - 1)
      window.requestAnimationFrame(update_frame);
  }

  function put_boids(scene, boids) {
    const R = decode(boids['R']);
    const R_shape = boids['R_shape'];
    const theta = decode(boids['theta']);
    const theta_shape = boids['theta_shape'];

    function index(i, b, xy) {
      return i * R_shape[1] * R_shape[2] + b * R_shape[2] + xy; 
    }

    var steps = R_shape[0];
    var boids = R_shape[1];
    var dimensions = R_shape[2];

    if(dimensions != 2) {
      alert('Can only deal with two-dimensional data.')
    }

    // First flatten the data.
    var buffer_data = new Float32Array(boids * 6);
    var size = 8.0;
    for (var i = 0 ; i < steps ; i++) {
      var buffer = gl.createBuffer();
      for (var b = 0 ; b < boids ; b++) {
        var xi = index(i, b, 0);
        var yi = index(i, b, 1);
        var ti = i * boids + b;
        var Nx = size * Math.cos(theta[ti]); //N[xi];
        var Ny = size * Math.sin(theta[ti]); //N[yi];
        buffer_data.set([
          R[xi] + Nx, R[yi] + Ny,
          R[xi] - Nx - 0.5 * Ny, R[yi] - Ny + 0.5 * Nx,
          R[xi] - Nx + 0.5 * Ny, R[yi] - Ny - 0.5 * Nx,             
        ], b * 6);
      }
      gl.bindBuffer(gl.ARRAY_BUFFER, buffer);
      gl.bufferData(gl.ARRAY_BUFFER, buffer_data, gl.STATIC_DRAW);

      scene.boid_buffer.push(buffer);
    }
    scene.boid_vertex_count = boids * 3;
    scene.frame_count += steps;
    return scene;
  }

  function put_predators(scene, boids) {
    // TODO: Unify this with the put_boids function.
    const R = decode(boids['R']);
    const R_shape = boids['R_shape'];
    const theta = decode(boids['theta']);
    const theta_shape = boids['theta_shape'];

    function index(i, b, xy) {
      return i * R_shape[1] * R_shape[2] + b * R_shape[2] + xy; 
    }

    var steps = R_shape[0];
    var boids = R_shape[1];
    var dimensions = R_shape[2];

    if(dimensions != 2) {
      alert('Can only deal with two-dimensional data.')
    }

    // First flatten the data.
    var buffer_data = new Float32Array(boids * 6);
    var size = 18.0;
    for (var i = 0 ; i < steps ; i++) {
      var buffer = gl.createBuffer();
      for (var b = 0 ; b < boids ; b++) {
        var xi = index(i, b, 0);
        var yi = index(i, b, 1);
        var ti = theta_shape[1] * i + b;
        var Nx = size * Math.cos(theta[ti]);
        var Ny = size * Math.sin(theta[ti]);
        buffer_data.set([
          R[xi] + Nx, R[yi] + Ny,
          R[xi] - Nx - 0.5 * Ny, R[yi] - Ny + 0.5 * Nx,
          R[xi] - Nx + 0.5 * Ny, R[yi] - Ny - 0.5 * Nx,             
        ], b * 6);
      }
      gl.bindBuffer(gl.ARRAY_BUFFER, buffer);
      gl.bufferData(gl.ARRAY_BUFFER, buffer_data, gl.STATIC_DRAW);

      scene.predator_buffer.push(buffer);
    }
    scene.predator_vertex_count = boids * 3;
    return scene;
  }

  function put_obstacle_disk(scene, disk) {
    const R = decode(disk.R);
    const R_shape = disk.R_shape;
    const radius = decode(disk.D);
    const radius_shape = disk.D_shape;

    const disk_count = R_shape[0];
    const dimensions = R_shape[1];
    if (dimensions != 2) {
        alert('Can only handle two-dimensional data.');
    }
    if (radius_shape[0] != disk_count) {
        alert('Inconsistent disk radius count found.');
    }
    const segments = 32;

    function index(o, xy) {
        return o * R_shape[1] + xy;
    }

    // TODO(schsam): Use index buffers here.
    var buffer_data = new Float32Array(disk_count * segments * 6);
    for (var i = 0 ; i < disk_count ; i++) {
      var xi = index(i, 0);
      var yi = index(i, 1);
      for (var s = 0 ; s < segments ; s++) {
        const th = 2 * s / segments * Math.PI;
        const th_p = 2 * (s + 1) / segments * Math.PI;
        const rad = radius[i] * 0.8;
        buffer_data.set([
          R[xi], R[yi],
          R[xi] + rad * Math.cos(th), R[yi] + rad * Math.sin(th),
          R[xi] + rad * Math.cos(th_p), R[yi] + rad * Math.sin(th_p),
        ], i * segments * 6 + s * 6);
      }
    }
    var buffer = gl.createBuffer();
    gl.bindBuffer(gl.ARRAY_BUFFER, buffer);
    gl.bufferData(gl.ARRAY_BUFFER, buffer_data, gl.STATIC_DRAW);
    scene.disk_vertex_count = disk_count * segments * 3;
    scene.disk_buffer = buffer;
    return scene;
  }

  // SHADER CODE

  const VERTEX_SHADER_SOURCE_2D = `
    // Vertex Shader Program.
    attribute vec2 vertex_position;
    
    uniform vec2 screen_position;
    uniform vec2 screen_size;

    void main() {
      vec2 v = (vertex_position - screen_position) / screen_size;
      gl_Position = vec4(v, 0.0, 1.0);
    }
  `;

  const FRAGMENT_SHADER_SOURCE_2D = `
    precision mediump float;

    uniform vec4 color;

    void main() {
      gl_FragColor = color;
    }
  `;

  function initialize_shader(
    gl, vertex_shader_source, fragment_shader_source) {

    const vertex_shader = compile_shader(
      gl, gl.VERTEX_SHADER, vertex_shader_source);
    const fragment_shader = compile_shader(
      gl, gl.FRAGMENT_SHADER, fragment_shader_source);

    const shader_program = gl.createProgram();
    gl.attachShader(shader_program, vertex_shader);
    gl.attachShader(shader_program, fragment_shader);
    gl.linkProgram(shader_program);

    if (!gl.getProgramParameter(shader_program, gl.LINK_STATUS)) {
      alert(
        'Unable to initialize shader program: ' + 
        gl.getProgramInfoLog(shader_program)
        );
        return null;
    }
    return shader_program;
  }

  function compile_shader(gl, type, source) {
    const shader = gl.createShader(type);
    gl.shaderSource(shader, source);
    gl.compileShader(shader);

    if (!gl.getShaderParameter(shader, gl.COMPILE_STATUS)) {
      alert('An error occured compiling shader: ' + gl.getShaderInfoLog(shader));
      gl.deleteShader(shader);
      return null;
    }

    return shader;
  }

  // SERIALIZATION UTILITIES
  function decode(sBase64, nBlocksSize) {
    var chrs = atob(atob(sBase64));
    var array = new Uint8Array(new ArrayBuffer(chrs.length));

    for(var i = 0 ; i < chrs.length ; i++) {
      array[i] = chrs.charCodeAt(i);
    }

    return new Float32Array(array.buffer);
  }

  // RUN CELL

  load_simulation();
  gl_and_shader = initialize_gl();
  var gl = gl_and_shader.gl;
  var shader = gl_and_shader.shader;
  update_frame();
</script>
''')

def encode(R):
  return base64.b64encode(onp.array(R, onp.float32).tobytes())

def render(box_size, states, obstacles=None, predators=None):
  if isinstance(states, Boids):
    R = np.reshape(states.R, (1,) + states.R.shape)
    theta = np.reshape(states.theta, (1,) + states.theta.shape)
  elif isinstance(states, list):
    if all([isinstance(x, Boids) for x in states]):
      R, theta = zip(*states)
      R = onp.stack(R)
      theta = onp.stack(theta)    
  
  if isinstance(predators, list):
    R_predators, theta_predators, *_ = zip(*predators)
    R_predators = onp.stack(R_predators)
    theta_predators = onp.stack(theta_predators)

  def get_boid_states(start, end):
    R_, theta_ = R[start:end], theta[start:end]
    return IPython.display.JSON(data={
        "R_shape": R_.shape,
        "R": encode(R_), 
        "theta_shape": theta_.shape,
        "theta": encode(theta_)
        })
  output.register_callback('notebook.GetBoidStates', get_boid_states)

  def get_obstacles():
    if obstacles is None:
      return IPython.display.JSON(data={})
    else:
      return IPython.display.JSON(data={
          'Disk': {
              'R': encode(obstacles.R),
              'R_shape': obstacles.R.shape,
              'D': encode(obstacles.D),
              'D_shape': obstacles.D.shape
          }
      })
  output.register_callback('notebook.GetObstacles', get_obstacles)

  def get_predators():
    if predators is None:
      return IPython.display.JSON(data={})
    else:
      return IPython.display.JSON(data={
          'R': encode(R_predators),
          'R_shape': R_predators.shape,
          'theta': encode(theta_predators),
          'theta_shape': theta_predators.shape
      })
  output.register_callback('notebook.GetPredators', get_predators)

  def get_simulation_info():
    return IPython.display.JSON(data={
        'frames': R.shape[0],
        'box_size': box_size
        })
  output.register_callback('notebook.GetSimulationInfo', get_simulation_info)

  return renderer_code

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 33.0 MB/s eta 0:00:00


# Flocks, Herds, and Schools: A Distributed Behavioral Model

This demo is based on the following paper, ["Flocks, Herds, and Schools: A Distributed Behavioral Model"]((https://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=E252054B1C02D387E8C20827CB414543?doi=10.1.1.103.7187&rep=rep1&type=pdf)) published by C. W. Reynolds in SIGGRAPH 1987.

In nature there are many examples in which large numbers of animals exhibit complex collective motion (schools of fish, flocks of birds, herds of horses, colonies of ants). In his seminal paper, Reynolds introduces a model of such collective behavior (henceforth refered to as "flocking") based on simple rules that can be computed locally for each entity (referred to as a "boid") in the flock based on its environment.

Note, we recommend running this notebook in "Dark" mode.

## Boids

Reynolds is interested in simulating entities that are described by a position, $R$, and an orientation, $\theta$. We can define a Boids type that stores data for a collection of boids as two arrays. `R` is an `ndarray` of shape `[boid_count, spatial_dimension]` and `theta` is an ndarray of shape `[boid_count]`. An individual boid is an index into these arrays. It will often be useful to refer to the vector orientation of the boid $N = (\cos\theta, \sin\theta)$.

In [ ]:
Boids = namedtuple('Boids', ['R', 'theta'])

We can instantiate a collection of boids randomly in a box of side length $L$. We will use [periodic boundary conditions](https://en.wikipedia.org/wiki/Periodic_boundary_conditions) for our simulation which means that boids will be able to wrap around the sides of the box. To do this we will use the `space.periodic` command in [JAX, MD](https://github.com/google/jax-md#spaces-spacepy).

In [ ]:
# Simulation Parameters:
box_size = 800.0  # A float specifying the side-length of the box.
boid_count = 200  # An integer specifying the number of boids.
dim = 2  # The spatial dimension in which we are simulating.

# Create RNG state to draw random numbers (see LINK).
rng = random.PRNGKey(0)

# Define periodic boundary conditions.
displacement, shift = space.periodic(box_size)

# Initialize the boids.
rng, R_rng, theta_rng = random.split(rng, 3)

boids = Boids(
  R = box_size * random.uniform(R_rng, (boid_count, dim)),
  theta = random.uniform(theta_rng, (boid_count,), maxval=2. * np.pi)
)

In [ ]:
display(render(box_size, boids))

## Dynamics

Each boid focuses on its local neighborhood (position r). Reynolds then proposes three simple, local, rules that boids might try to follow:


1.   **Steering:** Boids will try to align themselves in the direction of their neighbors.
2.   **Repulsion:** Boids will avoid colliding with their neighbors.
3.   **Adhesion:** Boids will try to move towards the center of mass of their neighbors.

\

The boids move at a fixed speed, $v$, along whatever direction they are pointing. 



\
An `update` function takes a boid's state to a new boids state.

In [ ]:
@vmap
def normal(theta):
  return np.array([np.cos(theta), np.sin(theta)])

def dynamics(energy_fn, dt, speed):
  @jit
  def update(_, state):
    R, theta = state['boids']

    dstate = quantity.force(energy_fn)(state)
    dR, dtheta = dstate['boids']
    n = normal(state['boids'].theta)

    state['boids'] = Boids(shift(R, dt * (speed * n + dR)), 
                           theta + dt * dtheta)

    return state

  return update

Now we can run a simulation and save the boid positions to a `boids_buffer` which will just be a list.

In [ ]:
update = dynamics(energy_fn=lambda state: 0., dt=1e-1, speed=1.)

boids_buffer = []

state = {
    'boids': boids
}

for i in ProgressIter(range(400)):
  state = lax.fori_loop(0, 50, update, state)
  boids_buffer += [state['boids']]

display(render(box_size, boids_buffer))

### Steering

While the above simulation works and our boids are moving happily along, it is not terribly interesting. The first thing that we can add to this simulation is the steering rule.

Boids should only interact with nearby boids. To do this, we introduce a cutoff $D_{\text{Align}}$ and ignore pairs of boids such that $\|\Delta R_{ij}\| > D_{\text{Align}}$ where $\Delta R_{ij} = R_i - R_j$. To make it so boids react smoothly we will have the energy start out at zero when $\|R_i - R_j\| = D_{\text{Align}}$ and increase smoothly as they get closer. Together, these simple ideas lead us to the following proposal,

$$\epsilon_{\text{Align}}(\Delta R_{ij}, \hat N_i, \hat N_j) = \begin{cases}\frac{J_{\text{Align}}}\alpha\left (1 - \frac{\|\Delta R_{ij}\|}{D_{\text{Align}}}\right)^\alpha(1 - \hat N_1 \cdot \hat N_2)^2 & \text{if $\|\Delta R_{ij}\| < D$}\\ 0 & \text{otherwise}\end{cases}$$

This energy will be maximized when $N_1$ and $N_2$ are anti-aligned and minimized when $N_1 = N_2$. In general, we would like our boids to turn to align themselves with their neighbors rather than shift their centers to move apart. Therefore, we'll insert a stop-gradient into the displacement.

In [ ]:
def align_fn(dR, N_1, N_2, J_align, D_align, alpha):
  dR = lax.stop_gradient(dR)
  dr = space.distance(dR) / D_align
  energy = J_align / alpha * (1. - dr) ** alpha * (1 - np.dot(N_1, N_2)) ** 2
  return np.where(dr < 1.0, energy, 0.)

We can now run our simulation with the steering energy alone.

In [ ]:
def energy_fn(state):
  boids = state['boids']
  E_align = partial(align_fn, J_align=0.5, D_align=45., alpha=3.)
  # Map the align energy over all pairs of boids. While both applications
  # of vmap map over the displacement matrix, each acts on only one normal.
  E_align = vmap(vmap(E_align, (0, None, 0)), (0, 0, None))

  dR = space.map_product(displacement)(boids.R, boids.R)
  N = normal(boids.theta)

  return 0.5 * np.sum(E_align(dR, N, N))

update = dynamics(energy_fn=energy_fn, dt=1e-1, speed=1.)

boids_buffer = []

state = {
    'boids': boids
}

for i in ProgressIter(range(400)):
  state = lax.fori_loop(0, 50, update, state)
  boids_buffer += [state['boids']]

display(render(box_size, boids_buffer))

### Repulsion

We can incorporate a repulsion rule that will keep the boids from bumping into one another and overlapping. This will help them to form a flock with some volume rather than collapsing together. To this end, imagine a very simple model of boids that push away from one another if they get within a distance $D_{\text{Avoid}}$ and otherwise don't repel. We can use a simple energy similar to Alignment but without any angular dependence,

$$
\epsilon_{\text{Avoid}}(\Delta R_{ij}) = \begin{cases}\frac{J_{\text{Avoid}}}{\alpha}\left(1 - \frac{||\Delta R_{ij}||}{D_{\text{Avoid}}}\right)^\alpha & ||\Delta R_{ij}||<D_{\text{Avoid}} \\ 0 & \text{otherwise}\end{cases}
$$

In [ ]:
def avoid_fn(dR, J_avoid, D_avoid, alpha):
  dr = space.distance(dR) / D_avoid
  return np.where(dr < 1., 
                  J_avoid / alpha * (1 - dr) ** alpha, 
                  0.)

We can now run a version of our simulation with both steering and repulsion. 

In [ ]:
def energy_fn(state):
  boids = state['boids']
  
  # Old Steering Code
  E_align = partial(align_fn, J_align=1., D_align=45., alpha=3.)
  E_align = vmap(vmap(E_align, (0, None, 0)), (0, 0, None))

  # New Repulsion Code
  E_avoid = partial(avoid_fn, J_avoid=25., D_avoid=30., alpha=3.)
  E_avoid = vmap(vmap(E_avoid))
  #

  dR = space.map_product(displacement)(boids.R, boids.R)
  N = normal(boids.theta)

  return 0.5 * np.sum(E_align(dR, N, N) + E_avoid(dR))

update = dynamics(energy_fn=energy_fn, dt=1e-1, speed=1.)

boids_buffer = []

state = {
    'boids': boids
}

for i in ProgressIter(range(400)):
  state = lax.fori_loop(0, 50, update, state)
  boids_buffer += [state['boids']]

display(render(box_size,  boids_buffer))

The avoidance term in the energy stops the boids from collapsing on top of one another.

### Adhesion



The final piece of Reynolds' boid model is adhesion. Notice that in the above simulation, the boids tend to move in the same direction but they also often drift apart. To make the boids maintain a more compact arrangement, we add a adhesion term to the energy. 

The goal of the adhesion term is to align boids towards the center of mass of their neighbors. Given a boid, $i$, we can compute the center of mass position of its neighbors as,

$$
\Delta R_i = \frac 1{|\mathcal N|} \sum_{j\in\mathcal N}\Delta R_{ij}
$$

where we have let $\mathcal N$ be the set of boids such that $||\Delta R_{ij}|| < D_{\text{Cohesion}}$. 

Given the center of mass displacements, we can define a reasonable adhesion energy as,

$$
\epsilon_{Cohesion}\left(\widehat{\Delta R}_i, N_i\right) = \frac12J_{\text{Cohesion}}\left(1 - \widehat {\Delta R}_i\cdot N\right)^2
$$

where $\widehat{\Delta R}_i = \Delta R_i / ||\Delta R_i||$ is the normalized vector pointing in the direction of the center of mass. This function is minimized when the boid is pointing in the direction of the center of  mass.


In [ ]:
def cohesion_fn(dR, N, J_cohesion, D_cohesion, eps=1e-7):
  dR = lax.stop_gradient(dR)
  dr = np.linalg.norm(dR, axis=-1, keepdims=True)
  
  mask = dr < D_cohesion

  N_com = np.where(mask, 1.0, 0)
  dR_com = np.where(mask, dR, 0)
  dR_com = np.sum(dR_com, axis=1) / (np.sum(N_com, axis=1) + eps)
  dR_com = dR_com / np.linalg.norm(dR_com + eps, axis=1, keepdims=True)
  return f32(0.5) * J_cohesion * (1 - np.sum(dR_com * N, axis=1)) ** 2

In [ ]:
def energy_fn(state):
  boids = state['boids']
  
  E_align = partial(align_fn, J_align=1., D_align=45., alpha=3.)
  E_align = vmap(vmap(E_align, (0, None, 0)), (0, 0, None))

  E_avoid = partial(avoid_fn, J_avoid=25., D_avoid=30., alpha=3.)
  E_avoid = vmap(vmap(E_avoid))
  
  # New Adhesion Code
  E_cohesion = partial(cohesion_fn, J_cohesion=0.005, D_cohesion=40.)
  #

  dR = space.map_product(displacement)(boids.R, boids.R)
  N = normal(boids.theta)

  return (0.5 * np.sum(E_align(dR, N, N) + E_avoid(dR)) + 
          np.sum(E_cohesion(dR, N)))

update = dynamics(energy_fn=energy_fn, dt=1e-1, speed=1.)

boids_buffer = []

state = {
    'boids': boids
}

for i in ProgressIter(range(400)):
  state = lax.fori_loop(0, 50, update, state)
  boids_buffer += [state['boids']]

display(render(box_size,  boids_buffer))

Now the boids travel in tighter, more cohesive, packs. By tuning the range of the cohesive interaction and its strength you can change how strongly the boids attempt to stick together. However, if we raise it too high it can have some undesireable consequences.

In [ ]:
def energy_fn(state):
  boids = state['boids']
  
  E_align = partial(align_fn, J_align=1., D_align=45., alpha=3.)
  E_align = vmap(vmap(E_align, (0, None, 0)), (0, 0, None))

  E_avoid = partial(avoid_fn, J_avoid=25., D_avoid=30., alpha=3.)
  E_avoid = vmap(vmap(E_avoid))

  E_cohesion = partial(cohesion_fn, J_cohesion=0.1, D_cohesion=40.)  # Raised to 0.05.

  dR = space.map_product(displacement)(boids.R, boids.R)
  N = normal(boids.theta)

  return (0.5 * np.sum(E_align(dR, N, N) + E_avoid(dR)) + 
          np.sum(E_cohesion(dR, N)))

update = dynamics(energy_fn=energy_fn, dt=1e-1, speed=1.)

boids_buffer = []

state = {
    'boids': boids
}

for i in ProgressIter(range(400)):
  state = lax.fori_loop(0, 50, update, state)
  boids_buffer += [state['boids']]

display(render(box_size,  boids_buffer))